# <font color='blue'>Data Science Academy</font>
# <font color='blue'>Deep Learning Para Aplicações de IA com PyTorch e Lightning</font>

## <font color='blue'>Mini-Projeto 3 - Deploy</font>
## <font color='blue'>Fine-Tuning de Modelo LLM Para Tarefa Específica e Deploy de Web App com Gradio</font>

![DSA](imagens/MP3.png)

## Instalando e Carregando os Pacotes

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.10.9


In [2]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# !pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

In [3]:
%env TF_CPP_MIN_LOG_LEVEL=3

env: TF_CPP_MIN_LOG_LEVEL=3


In [4]:
# https://pypi.org/project/gradio/
!pip install -q gradio

In [5]:
# Imports
import torch
import gradio as gr
from transformers import AutoModelForCausalLM

In [6]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" --iversions

Author: Data Science Academy

torch : 2.0.1
gradio: 3.39.0



In [7]:
# Carrega o modelo
modelo_llm = AutoModelForCausalLM.from_pretrained("modelos/modelo_final")

In [8]:
# Definindo uma classe chamada NumberTokenizer, que é usada para tokenizar os números
class DSATokenizer:
    
    # Método construtor da classe, que é executado quando um objeto dessa classe é criado
    def __init__(self, numbers_qty = 10):
        
        # Lista de tokens possíveis que o tokenizador pode encontrar
        vocab = ['+', '=', '-1', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
        
        # Definindo a quantidade de números que o tokenizador pode lidar
        self.numbers_qty = numbers_qty
        
        # Definindo o token de preenchimento (padding)
        self.pad_token = '-1'
        
        # Criando um dicionário que mapeia cada token para um índice único
        self.encoder = {str(v):i for i,v in enumerate(vocab)}
        
        # Criando um dicionário que mapeia cada índice único de volta ao token correspondente
        self.decoder = {i:str(v) for i,v in enumerate(vocab)}
        
        # Obtendo o índice do token de preenchimento no encoder
        self.pad_token_id = self.encoder[self.pad_token]

    # Método para decodificar uma lista de IDs de token de volta para uma string
    def decode(self, token_ids):
        return ' '.join(self.decoder[t] for t in token_ids)

    # Método que é chamado quando o objeto da classe é invocado como uma função
    def __call__(self, text):
        # Dividindo o texto em tokens individuais e retornando uma lista dos IDs correspondentes
        return [self.encoder[t] for t in text.split()]

In [9]:
# Cria o objeto
tokenizer = DSATokenizer(13)

In [10]:
# Definindo a função gera_solution com três parâmetros: input, solution_length e model
def faz_previsao(entrada, solution_length = 6, model = modelo_llm):

    # Colocando o modelo em modo de avaliação. 
    model.eval()

    # Convertendo a entrada (string) em tensor utilizando o tokenizer. 
    # O tensor é uma estrutura de dados que o modelo de aprendizado de máquina pode processar.
    entrada = torch.tensor(tokenizer(entrada))

    # Iniciando uma lista vazia para armazenar a solução
    solution = []

    # Loop que gera a solução de comprimento solution_length
    for i in range(solution_length):

        # Alimentando a entrada atual ao modelo e obtendo a saída
        saida = model(entrada)

        # Pegando o índice do maior valor no último conjunto de logits (log-odds) da saída, 
        # que é a previsão do modelo para o próximo token
        predicted = saida.logits[-1].argmax()

        # Concatenando a previsão atual com a entrada atual. 
        # Isso servirá como a nova entrada para a próxima iteração.
        entrada = torch.cat((entrada, predicted.unsqueeze(0)), dim = 0)

        # Adicionando a previsão atual à lista de soluções e convertendo o tensor em um número Python padrão
        solution.append(predicted.cpu().item())

    # Decodificando a lista de soluções para obter a string de saída e retornando-a
    return tokenizer.decode(solution)

In [11]:
# Testa a função
faz_previsao('3 + 5 =', solution_length = 2)

'0 8'

In [12]:
# Função para retornar a função que faz a previsão
def funcsolve(entrada):
    return faz_previsao(entrada, solution_length = 2)

In [13]:
# Cria a web app
webapp = gr.Interface(fn = funcsolve, 
                      inputs = [gr.Textbox(label = "Dados de Entrada", 
                                           lines = 1, 
                                           info = "Os dados devem estar na forma: '1 + 2 =' com um único espaço entre cada caractere e apenas números de um dígito são permitidos.")],
                      outputs = [gr.Textbox(label = "Resultado (Previsão do Modelo)", lines = 1)],
                      title = "Deploy de LLM Após o Fine-Tuning",
                      description = "Digite os dados de entrada e clique no botão Submit para o modelo fazer a previsão.",
                      examples = ["5 + 3 =", "2 + 9 ="]) 

In [14]:
webapp.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/Users/dmpm/anaconda3/lib/python3.10/site-packages/gradio/routes.py", line 442, in run_predict
    output = await app.get_blocks().process_api(
  File "/Users/dmpm/anaconda3/lib/python3.10/site-packages/gradio/blocks.py", line 1392, in process_api
    result = await self.call_function(
  File "/Users/dmpm/anaconda3/lib/python3.10/site-packages/gradio/blocks.py", line 1097, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/Users/dmpm/anaconda3/lib/python3.10/site-packages/anyio/to_thread.py", line 28, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(func, *args, cancellable=cancellable,
  File "/Users/dmpm/anaconda3/lib/python3.10/site-packages/anyio/_backends/_asyncio.py", line 818, in run_sync_in_worker_thread
    return await future
  File "/Users/dmpm/anaconda3/lib/python3.10/site-packages/anyio/_backends/_asyncio.py", line 754, in run
    result = context.run(func, *args)
  File "/Users/dmpm/an

# Fim